## Create The ONNX File

In [1]:
from special_neurons import get_most_negative_sets
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import re

model_name = 'EleutherAI/pythia-160m'
model_name = 'EleutherAI/pythia-70m'
# model_name = 'EleutherAI/gpt-neo-1.3B'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
model_og = AutoModelForCausalLM.from_pretrained(model_name).to(device)
most_neg = get_most_negative_sets(model_og)
most_neg[0].prev_layer_name, most_neg[0].linear_layer_name

/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.1) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


('gpt_neox.layers.0.mlp', 'gpt_neox.layers.0.mlp.dense_h_to_4h')

In [2]:
model_og

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 512)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-5): 6 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
          (dense): Linear(in_features=512, out_features=512, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
          (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
          (a

In [36]:
def get_input_after_tokenizer(inp: str):
		tokenizer = AutoTokenizer.from_pretrained(model_name)
		inp_out = tokenizer(inp, return_tensors="pt")
		print(inp_out)
		inputs = inp_out['input_ids'].squeeze(0)
		# print(inputs, inputs.shape)
		one_hot = torch.zeros((inputs.shape[0], tokenizer.vocab_size), dtype=torch.int)
		for i in range(inputs.shape[0]):
			one_hot[i, inputs[i]] = 1
		# one_hot[inputs['input_ids'][0, 0]] = 1
		return one_hot#, inp_out['attention_mask']
get_input_after_tokenizer('hello world')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


{'input_ids': tensor([[25521,  1533]]), 'attention_mask': tensor([[1, 1]])}


tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], dtype=torch.int32)

In [39]:
N_TOKENS = 10
attention_mask = torch.ones((1, N_TOKENS), dtype=torch.int)


class FixedAttentionMask(torch.nn.Module):
  	
    def __init__(self, *args, **kwargs) -> None:
       super().__init__(*args, **kwargs)
       self.attn = model_og.gpt_neox.layers[0].attention

    def forward(self, x):
      return self.attn(x, attention_mask=attention_mask)

# G
# TODO: add residuals?
model_sel = torch.nn.Sequential(
    model_og.gpt_neox.embed_in,
    # model_og.gpt_neox.emb_dropout, # we have p = 0.0 and thus useless
    model_og.gpt_neox.layers[0].input_layernorm,
    FixedAttentionMask(),
    # TODO: VERIFY THIS JAZZ
)

model_sel

Sequential(
  (0): Embedding(50304, 512)
  (1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  (2): FixedAttentionMask(
    (attn): GPTNeoXAttention(
      (rotary_emb): GPTNeoXRotaryEmbedding()
      (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
      (dense): Linear(in_features=512, out_features=512, bias=True)
      (attention_dropout): Dropout(p=0.0, inplace=False)
    )
  )
)

In [33]:
inps_one_hot = get_input_after_tokenizer("hello world")
inps_one_hot.shape, inps_one_hot.nonzero(), model_sel(inps_one_hot)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


TypeError: GPTNeoXAttention.forward() missing 2 required positional arguments: 'attention_mask' and 'position_ids'

In [ ]:

torch.onnx.export(model_sel, torch.randn(1, 1).to(device),
                  'model_sel.onnx', verbose=True, opset_version=12)

## Use Mariboupy

In [ ]:
import sys
import numpy as np

## %
# Path to Marabou folder if you did not export it

# TODO: this is a hack, fix it
# TODO: add Marabou installation details to readme
# sys.path.append('/home/lev/code/research/softmax_bound/bounding-softmax/Marabou')
from maraboupy import Marabou